# Time Series Model for Birmingham Parking Occupancy Using Python and ARIMA Part 2
### David Lowe
### October 21, 2020

Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. https://machinelearningmastery.com/

SUMMARY: This project aims to construct a time series prediction model and document the end-to-end steps using a template. The Birmingham Parking Occupancy dataset is a time series situation where we are trying to forecast future outcomes based on past data points.

INTRODUCTION: The problem is to forecast the hourly number of parking occupancy for a parking facility in Birmingham. The dataset describes a time-series of parking occupancy over three months between October 2016 and December 2016, and there are 1834 hourly observations. We used the first 90% of the observations for training various models while holding back the remaining observations for validating the final model.

From iteration Part1, we trained and validated an ARIMA model using just one facility, BHMBCCMKT01, within the dataset.

In this Part2 iteration, we will train and validate an ARIMA model for each one of the facilities within the dataset.

ANALYSIS: The baseline prediction (or persistence) for the parking facility BHMBCCMKT01 resulted in an RMSE of 46. After performing a grid search for the most optimal ARIMA parameters, the final ARIMA non-seasonal order was (2, 0, 1) with the seasonal order (2, 0, 0, 24). Furthermore, the chosen model processed the validation data with an RMSE of 22, which was better than the baseline model as expected.

Parking structure: BHMBCCPST01
RMSE for the persistent model is: 38
Final Non-season order: (0, 0, 1) Final Seasonal Order: (1, 0, 1, 24)
RMSE from the validation data is: 20

Parking structure: BHMBCCSNH01
RMSE for the persistent model is: 157
Final Non-season order: (2, 0, 1) Final Seasonal Order: (0, 0, 2, 24)
RMSE from the validation data is: 75

Parking structure: BHMBCCTHL01
RMSE for the persistent model is: 84
Final Non-season order: (0, 0, 0) Final Seasonal Order: (1, 0, 1, 24)
RMSE from the validation data is: 24

Parking structure: BHMNCPPLS01
RMSE for the persistent model is: 32
Final Non-season order: (4, 0, 0) Final Seasonal Order: (1, 0, 0, 24)
RMSE from the validation data is: 16

Parking structure: BHMBRCBRG02
RMSE for the persistent model is: 189
Final Non-season order: (0, 1, 3) Final Seasonal Order: (0, 0, 2, 24)
RMSE from the validation data is: 95

Parking structure: BHMBRCBRG03
RMSE for the persistent model is: 78
Final Non-season order: (2, 1, 0) Final Seasonal Order: (0, 0, 2, 24)
RMSE from the validation data is: 41

Parking structure: BHMBRTARC01
RMSE for the persistent model is: 109
Final Non-season order: (1, 0, 0) Final Seasonal Order: (1, 0, 0, 24)
RMSE from the validation data is: 120

Parking structure: BHMEURBRD01
RMSE for the persistent model is: 77
Final Non-season order: (1, 0, 4) Final Seasonal Order: (2, 0, 1, 24)
RMSE from the validation data is: 24

CONCLUSION: For this dataset, the chosen ARIMA model achieved a satisfactory result, and we should consider using ARIMA for further modeling.

Dataset Used: Parking Birmingham Data Set

Dataset ML Model: Time series forecast with numerical attribute

Dataset Reference: https://archive.ics.uci.edu/ml/datasets/Parking+Birmingham

A time series predictive modeling project generally can be broken down into about five major tasks:

1. Define Problem and Acquire Data
2. Inspect and Explore Data
3. Clean and Pre-Process Data
4. Fit and Evaluate Models
5. Finalize Model and Make Predictions

## Task 1: Define Problem and Acquire Data

In [1]:
# Install the necessary packages for Colab
# !pip install python-dotenv PyMySQL

In [2]:
# Retrieve the GPU information from Colab
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#     print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
#     print('and then re-execute this cell.')
# else:
#     print(gpu_info)

In [3]:
# Retrieve the memory configuration from Colab
# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#     print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
#     print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
#     print('re-execute this cell.')
# else:
#     print('You are using a high-RAM runtime!')

In [4]:
# Retrieve the CPU information
# ncpu = !nproc
# print("The number of available CPUs is:", ncpu[0])

### 1.a) Load Libraries

In [5]:
# Create the random seed number for reproducible results
seedNum = 888

In [6]:
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import pmdarima as pm
import os
import sys
import math
import boto3
from datetime import datetime
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
# from statsmodels.tsa.stattools import adfuller
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.graphics.tsaplots import plot_acf
# from statsmodels.graphics.tsaplots import plot_pacf
# import pandas_datareader.data as web

### 1.b) Set up the controlling parameters and functions

In [7]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the number of CPU cores available for multi-thread processing
n_jobs = 1

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set Pandas options
pd.set_option("display.max_rows", 500)
pd.set_option("display.width", 140)

In [8]:
# # Set up the parent directory location for loading the dotenv files
# useGDrive = False
# if useGDrive:
#     # Mount Google Drive locally for storing files
#     from google.colab import drive
#     drive.mount('/content/gdrive')
#     gdrivePrefix = '/content/gdrive/My Drive/Colab_Downloads/'
#     env_path = '/content/gdrive/My Drive/Colab Notebooks/'
#     dotenv_path = env_path + "python_script.env"
#     load_dotenv(dotenv_path=dotenv_path)

# # Set up the dotenv file for retrieving environment variables
# useLocalPC = False
# if useLocalPC:
#     env_path = "/Users/david/PycharmProjects/"
#     dotenv_path = env_path + "python_script.env"
#     load_dotenv(dotenv_path=dotenv_path)

In [9]:
# Set up the email notification function
def status_notify(msg_text):
    access_key = os.environ.get('SNS_ACCESS_KEY')
    secret_key = os.environ.get('SNS_SECRET_KEY')
    aws_region = os.environ.get('SNS_AWS_REGION')
    topic_arn = os.environ.get('SNS_TOPIC_ARN')
    if (access_key is None) or (secret_key is None) or (aws_region is None):
        sys.exit("Incomplete notification setup info. Script Processing Aborted!!!")
    sns = boto3.client('sns', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=aws_region)
    response = sns.publish(TopicArn=topic_arn, Message=msg_text)
    if response['ResponseMetadata']['HTTPStatusCode'] != 200 :
        print('Status notification not OK with HTTP status code:', response['ResponseMetadata']['HTTPStatusCode'])

In [10]:
if notifyStatus: status_notify("ARIMA Task 1: Define Problem and Acquire Data has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

### 1.c) Acquire and Load the Data

In [11]:
# load the dataset and the necessary data structure
original_series = pd.read_csv('https://dainesanalytics.com/datasets/ucirvine-birmingham-parking-occupancy/dataset.csv', index_col='LastUpdated', parse_dates=True)

In [12]:
if notifyStatus: status_notify("ARIMA Task 1: Define Problem and Acquire Data completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

## Task 2: Inspect and Explore Data

In [13]:
if notifyStatus: status_notify("ARIMA Task 2: Inspect and Explore Data has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [14]:
# Omitted for this iteration of modeling

In [15]:
if notifyStatus: status_notify("ARIMA Task 2: Inspect and Explore Data completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

## Task 3: Clean and Pre-Process Data

In [16]:
if notifyStatus: status_notify("ARIMA Task 3: Clean and Pre-Process Data has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [17]:
# Omitted for this iteration of modeling

In [18]:
if notifyStatus: status_notify("ARIMA Task 3: Clean and Pre-Process Data completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

## Task 4: Fit and Evaluate Models

In [19]:
if notifyStatus: status_notify("ARIMA Task 4: Fit and Evaluate Models has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [20]:
# Prepare and evaluate a persistence model
def calculate_persistent_model(train_ts, test_ts):
    # walk-forward validation
    history = [x for x in train_ts]
    predictions = list()
    for i in range(len(test_ts)):
        yhat = history[-1]
        predictions.append(yhat)
        obs = test_ts[i]
        history.append(obs)

    # Calculate performance
    rmse = math.sqrt(mean_squared_error(test_ts, predictions))
    print('RMSE for the persistent model is: %.0f' % rmse)

In [21]:
def calculate_arima_model(train_ts):
    
    # Set the initial seasonal frequency parameter
    seasonal_freq = 24

    # Do an automated stepwise search of ARIMA parameters
    print('Searching optimal ARIMA parameters through stepwise search...')
    stepwise_results = pm.auto_arima(train_ts, seasonal = True, m = seasonal_freq, stepwise = True, trace = False, suppress_warnings = True, random_state = seedNum)
    
    # Set the ARIMA order parameters for validation and forecasting
    ns_order = stepwise_results.order
    ss_order = stepwise_results.seasonal_order
    print("Final Non-season order:", ns_order, 'Final Seasonal Order:', ss_order)
    return(ns_order, ss_order)

In [22]:
def validate_model_error(time_series, test_size, final_ns_order, final_ss_order):
    # Summarize residual errors for the final ARIMA model
    final_model = SARIMAX(time_series, order=final_ns_order, seasonal_order=final_ss_order)
    final_results = final_model.fit(disp = False)
#     print(final_results.summary())
#     final_results.plot_diagnostics(figsize=(12,12))
#     plt.show()

    validate_forecast = final_results.get_prediction(start = -test_size)
    mean_validate_forecast = validate_forecast.predicted_mean
    validate_confidence_intervals = validate_forecast.conf_int()

    # Occasionally some predicted values turned out to be < 0 but they should not be (e.g. rainfall, disease cases, etc.)
    # If we have those values, we will need to set them to 0
#     print(mean_validate_forecast[mean_validate_forecast < 0])
    mean_validate_forecast[mean_validate_forecast < 0] = 0

    # Evaluate RMSE for the validation data
    y_test = time_series[-test_size:]
    predictions = mean_validate_forecast.values
#     for i in range(y_test.shape[0]):
#         print(y_test.index[i], ' | ', y_test.iloc[i,0], ' | ', predictions[i])
    print('RMSE from the validation data is: %.0f' % math.sqrt(mean_squared_error(y_test, predictions)))

In [23]:
if notifyStatus: status_notify("ARIMA Task 4: Fit and Evaluate Models completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

## Section 5. Finalize Model and Make Predictions

In [24]:
if notifyStatus: status_notify("ARIMA Task 5: Finalize Model and Make Predictions has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [25]:
facilityCodes = ['BHMBCCMKT01', 'BHMBCCPST01', 'BHMBCCSNH01', 'BHMBCCTHL01', 'BHMNCPPLS01',
                 'BHMBRCBRG02', 'BHMBRCBRG03', 'BHMBRTARC01', 'BHMEURBRD01']

In [26]:
for targetSystemCode in facilityCodes:
    parking_data = original_series.copy()
    parking_data = parking_data[parking_data['SystemCodeNumber']==targetSystemCode]
    parking_data.drop(columns=['SystemCodeNumber','Capacity'], inplace=True)
    parking_data.sort_values(by=['LastUpdated'], ascending=True, inplace=True)
    parking_data = parking_data.resample('H').mean()
    parking_data['Occupancy'].fillna(0, inplace=True)
    
    X = parking_data.values
    X = X.astype('float32')
    train_pct = 0.90
    train_size = int(len(X) * train_pct)
    validate_size = len(X) - train_size
    train_data, validate_data = X[0:train_size], X[train_size:]

    print('Modeling the parking structure:', targetSystemCode)
    calculate_persistent_model(train_data, validate_data)
    nonseason_order, seasonal_order = calculate_arima_model(train_data)
    validate_model_error(parking_data, validate_size, nonseason_order, seasonal_order)
    print('Modeling of the parking structure:', targetSystemCode, 'completed!')
    print()

Modeling the parking structure: BHMBCCMKT01
RMSE for the persistent model is: 46
Searching optimal ARIMA parameters through stepwise search...
Final Non-season order: (2, 0, 1) Final Seasonal Order: (2, 0, 0, 24)
RMSE from the validation data is: 22
Modeling of the parking structure: BHMBCCMKT01 completed!

Modeling the parking structure: BHMBCCPST01
RMSE for the persistent model is: 38
Searching optimal ARIMA parameters through stepwise search...
Final Non-season order: (0, 0, 1) Final Seasonal Order: (1, 0, 1, 24)
RMSE from the validation data is: 20
Modeling of the parking structure: BHMBCCPST01 completed!

Modeling the parking structure: BHMBCCSNH01
RMSE for the persistent model is: 157
Searching optimal ARIMA parameters through stepwise search...
Final Non-season order: (2, 0, 1) Final Seasonal Order: (0, 0, 2, 24)
RMSE from the validation data is: 75
Modeling of the parking structure: BHMBCCSNH01 completed!

Modeling the parking structure: BHMBCCTHL01
RMSE for the persistent mode

In [27]:
if notifyStatus: status_notify("ARIMA Task 5: Finalize Model and Make Predictions completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [28]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 1:22:56.921662
